In [1]:
from modeling_qwen2_vl import Qwen2VLForConditionalGeneration
from processing_qwen2_vl import Qwen2VLProcessor
# from configuration_qwen2_vl import Qwen2VLConfig
# from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor
from qwen_vl_utils import process_vision_info
import time 

In [2]:
model_path = "/data/data1/syc/intern/wanshan/models/Qwen2-VL-2B-Instruct"
# model_path = "/data/data1/syc/intern/wanshan/models/showlab/ShowUI-2B_edited"
device = 'cuda:3'

In [33]:
min_pixel = 256*28*28
max_pixel = 1344*28*28
import numpy
# 1. Screenshot -> Graph
uigraph_train = True        # Enable ui graph during training
uigraph_test = True         # Enable ui graph during inference
uigraph_diff = 1            # Pixel difference used for constructing ui graph
uigraph_rand = False        # Enable random graph construction 
# 2. Graph -> Mask 
uimask_pre = True           # Prebuild patch selection mask in the preprocessor (not in model layers) for efficiency
uimask_ratio = 0.8         # Specify the percentage of patch tokens to skip per component
uimask_rand = False         # Enable random token selection instead of uniform selection

In [36]:
ratios = numpy.arange(start=0, stop=1, step=0.1)
visual_tokens = []
ratios

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

In [45]:

for ratio in ratios:
    print(ratio)
    processor = Qwen2VLProcessor.from_pretrained(
            model_path,
            min_pixels= min_pixel,
            max_pixels = max_pixel,
            uigraph_train=uigraph_train, uigraph_test=uigraph_test, uigraph_diff=uigraph_diff, uigraph_rand=uigraph_rand,
            uimask_pre=True, uimask_ratio=ratio, uimask_rand=uimask_rand,
            use_fast = True
        )

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": "./chrome.png",
                    "min_pixels": min_pixel,
                    "max_pixels": max_pixel,
                },
                {"type": "text", "text": "Describe this image."},
            ],
        }
    ]
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
        # vis_dir="./" # this folder to save visualization 
    )
    inputs = inputs.to(device)
    vs_tokens = inputs['select_mask'].sum() - 25 # text tokens = 25
    print(vs_tokens)
    visual_tokens.append(vs_tokens)

0.0
tensor(1296, device='cuda:3')
0.1
tensor(1184, device='cuda:3')
0.2
tensor(1067, device='cuda:3')
0.30000000000000004
tensor(951, device='cuda:3')
0.4
tensor(838, device='cuda:3')
0.5
tensor(721, device='cuda:3')
0.6000000000000001
tensor(608, device='cuda:3')
0.7000000000000001
tensor(495, device='cuda:3')
0.8
tensor(380, device='cuda:3')
0.9
tensor(268, device='cuda:3')


: 

In [42]:
1321 - 1296

25

In [41]:
inputs['patch_assign'].shape

torch.Size([1296])

In [39]:
visual_tokens

[tensor(1321, device='cuda:3'),
 tensor(1209, device='cuda:3'),
 tensor(1092, device='cuda:3'),
 tensor(976, device='cuda:3'),
 tensor(863, device='cuda:3'),
 tensor(746, device='cuda:3'),
 tensor(633, device='cuda:3'),
 tensor(520, device='cuda:3'),
 tensor(405, device='cuda:3'),
 tensor(293, device='cuda:3'),
 tensor(1321, device='cuda:3'),
 tensor(1209, device='cuda:3')]

In [22]:
inputs.image_grid_thw

tensor([[ 1, 54, 96]], device='cuda:3')

In [23]:
inputs.pixel_values.shape

torch.Size([5184, 1176])

In [24]:
inputs['select_mask'].sum()

tensor(405, device='cuda:3')

In [12]:
print(processor)

Qwen2VLProcessor:
- image_processor: Qwen2VLImageProcessor {
  "do_convert_rgb": true,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.48145466,
    0.4578275,
    0.40821073
  ],
  "image_processor_type": "Qwen2VLImageProcessor",
  "image_std": [
    0.26862954,
    0.26130258,
    0.27577711
  ],
  "max_pixels": 1053696,
  "merge_size": 2,
  "min_pixels": 200704,
  "patch_size": 14,
  "processor_class": "Qwen2VLProcessor",
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "longest_edge": 12845056,
    "shortest_edge": 3136
  },
  "temporal_patch_size": 2
}

- tokenizer: Qwen2TokenizerFast(name_or_path='/data/data1/syc/intern/wanshan/models/Qwen2-VL-2B-Instruct', vocab_size=151643, model_max_length=32768, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', 

In [4]:
inputs["patch_pos"].shape

torch.Size([1, 1321])

In [7]:
for key in inputs.keys():
    print(key)

input_ids
attention_mask
patch_pos
select_mask
pixel_values
image_grid_thw
patch_assign
patch_assign_sep
patch_assign_len


In [8]:
img_grid_thw = inputs['image_grid_thw']
img_grid_thw # [bsz, heigh // patch_size // merge_size, width // patch_size // merge_size]

tensor([[ 1, 54, 96]], device='cuda:2')

In [9]:
patch_pos = inputs['patch_pos']
patch_pos.shape # [bsz, seq_len]

torch.Size([1, 1321])

In [10]:
select_mask = inputs['select_mask']
select_mask.sum() # total selected tokens

tensor(746, device='cuda:2')

- Number of visual tokens

In [11]:
patch_assign = inputs['patch_assign']
print(f'patch_assign shape : {patch_assign.shape}') # [# visual tokens]

# verify number of visual tokens 
patch_pos = inputs['patch_pos']
print(patch_pos[patch_pos != -1].shape) # total visual tokens

from PIL import Image
img_path = './chrome.png'
img = Image.open(img_path)
# smart resize 
w, h = img.size
# Function to find the nearest multiple of 28
def nearest_multiple(value, base=28):
    return round(value / base) * base

# Compute the new dimensions
new_w = nearest_multiple(w, 28)
new_h = nearest_multiple(h, 28)
print(new_w // 28 * new_h // 28) 

patch_assign shape : torch.Size([1296])
torch.Size([1296])
2691


- Number of UI components

In [12]:
patch_assign_len = inputs['patch_assign_len']
patch_assign_len # total ui components

tensor([167], device='cuda:2')

# Model Generation

In [13]:
import torch
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_path, 
    torch_dtype = torch.bfloat16,
    device_map = device).eval()

NameError: name 'Qwen2VLForConditionalGeneration' is not defined

In [11]:
generated_ids = model.generate(**inputs, max_new_tokens=128, do_sample=True)

generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, 
)[0]

print(output_text)

patch_pos torch.Size([1, 980])
retain_mask tensor(559, device='cuda:2')
patch_pos torch.Size([1, 980])
retain_mask tensor(559, device='cuda:2')
patch_pos torch.Size([1, 980])
retain_mask tensor(559, device='cuda:2')
patch_pos torch.Size([1, 980])
retain_mask tensor(559, device='cuda:2')
patch_pos torch.Size([1, 980])
retain_mask tensor(559, device='cuda:2')
patch_pos torch.Size([1, 980])
retain_mask tensor(559, device='cuda:2')
patch_pos torch.Size([1, 980])
retain_mask tensor(559, device='cuda:2')
patch_pos torch.Size([1, 980])
retain_mask tensor(559, device='cuda:2')
patch_pos torch.Size([1, 980])
retain_mask tensor(559, device='cuda:2')
patch_pos torch.Size([1, 980])
retain_mask tensor(559, device='cuda:2')
patch_pos torch.Size([1, 980])
retain_mask tensor(559, device='cuda:2')
patch_pos torch.Size([1, 980])
retain_mask tensor(559, device='cuda:2')
patch_pos torch.Size([1, 980])
retain_mask tensor(559, device='cuda:2')
patch_pos torch.Size([1, 980])
retain_mask tensor(559, device='c